In [7]:

GYRO = False
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 4
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"physical_activity_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

# EDIT THE VALUES IN THE CELL ABOVE THEN PRESS RUN ALL

In [8]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


In [9]:
DATA_DIRECTORY = "./all_respeck"
RESPIRATORY_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    
    "standing&coughing",
    "standing&hyperventilating",
    
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
}

In [10]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in RESPIRATORY_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [11]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [12]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(RESPIRATORY_ACTIVITIES)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

there are 14340 tagged sequences in the dataset


In [13]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [14]:

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, RESPIRATORY_ACTIVITIES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/Task2/{MODEL_NAME}_{test_accuracy}")

Epoch 1/50
1148/1148 [==============================] - 8s 6ms/step - loss: 1.0963 - accuracy: 0.4244 - val_loss: 0.9704 - val_accuracy: 0.4623
Epoch 2/50
1148/1148 [==============================] - 6s 5ms/step - loss: 0.9955 - accuracy: 0.4443 - val_loss: 0.9494 - val_accuracy: 0.4589
Epoch 3/50
1148/1148 [==============================] - 6s 5ms/step - loss: 0.9636 - accuracy: 0.4648 - val_loss: 0.9086 - val_accuracy: 0.5265
Epoch 4/50
1148/1148 [==============================] - 6s 5ms/step - loss: 0.9299 - accuracy: 0.4952 - val_loss: 0.9134 - val_accuracy: 0.5363
Epoch 5/50
1148/1148 [==============================] - 6s 6ms/step - loss: 0.8026 - accuracy: 0.6241 - val_loss: 0.6764 - val_accuracy: 0.7141
Epoch 6/50
1148/1148 [==============================] - 6s 5ms/step - loss: 0.6223 - accuracy: 0.7374 - val_loss: 0.5197 - val_accuracy: 0.8033
Epoch 7/50
1148/1148 [==============================] - 6s 5ms/step - loss: 0.5292 - accuracy: 0.7741 - val_loss: 0.5380 - val_accuracy:

2023-11-05 03:38:28.213564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]
2023-11-05 03:38:28.724315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/Task2/physical_activity_epochs:50_batch:10_gyro:False_window:5_overlap:4.keras_0.910041868686676/assets


INFO:tensorflow:Assets written to: models/Task2/physical_activity_epochs:50_batch:10_gyro:False_window:5_overlap:4.keras_0.910041868686676/assets
